In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ase
import rmsd

In [3]:
from ase.visualize import view
from ase.optimize import BFGS

In [4]:
# import path
import sys
sys.path.append("tutorial") # go to parent dir

In [5]:
# tutorial modules
import calculators

Let us start with importing a molecule

In [6]:
atom_labels, coordinates = rmsd.get_coordinates_xyz("examples/ethanol.xyz")
molecule = ase.Atoms(atom_labels, coordinates)
view(molecule, viewer="x3d")

## Optimization using OQML forces

Doesn't seem very optimized, so let us optimize it. We do this by

In [7]:
calculator = calculators.get_calculator("_deploy_")

In [8]:
molecule.set_calculator(calculator)

In [9]:
dyn = BFGS(molecule)
dyn.run(fmax=0.5)

      Step     Time          Energy         fmax
BFGS:    0 22:37:46      -30.748595        2.9513
BFGS:    1 22:37:46      -31.014309        2.8324
BFGS:    2 22:37:46      -31.262773        3.6434
BFGS:    3 22:37:46      -31.505507        4.8465
BFGS:    4 22:37:46      -31.743424        6.6610
BFGS:    5 22:37:46      -31.971456        8.1748
BFGS:    6 22:37:46      -32.184914        9.2098
BFGS:    7 22:37:46      -32.382798        9.6392
BFGS:    8 22:37:47      -32.569092        9.3728
BFGS:    9 22:37:47      -32.734802        8.4763
BFGS:   10 22:37:47      -32.891674        7.8435
BFGS:   11 22:37:47      -33.054999        7.0145
BFGS:   12 22:37:47      -33.227541        5.9866
BFGS:   13 22:37:47      -33.403943        4.8442
BFGS:   14 22:37:47      -33.575563        3.6585
BFGS:   15 22:37:47      -33.719203        2.6356
BFGS:   16 22:37:47      -33.842170        1.7739
BFGS:   17 22:37:47      -33.948032        1.0753
BFGS:   18 22:37:47      -34.039315        0.7158
B

True

In [10]:
view(molecule, viewer="x3d")